
# 基于修改后的 MindQuantum 0.5.0 采用变分算法重构半导体双量子点单-三重态量子 CY 门

# 采用自定义的 MQLayer abs 来实现

In [4]:
import numpy as np
import sys
from numpy import kron
from mindquantum import *
from scipy.linalg import expm
import mindspore as ms
from mindspore import ops, Tensor
ms.context.set_context(mode=ms.context.PYNATIVE_MODE, device_target="CPU")
from mindspore.nn import Adam, TrainOneStepCell
from mindspore.common.parameter import Parameter
from mindspore.common.initializer import initializer  
ms.set_seed(1)
np.random.seed(1)

train_x = np.load('./src/cy_train_x.npy', allow_pickle=True)
eval_x = np.load('./src/cy_eval_x.npy', allow_pickle=True)
train_y = np.load('./src/cy_train_y.npy', allow_pickle=True)
eval_y = np.load('./src/cy_eval_y.npy', allow_pickle=True)

s_x = X.matrix()
s_z = Z.matrix()
one = I.matrix()
dt = np.pi/10

def _matrix_0(coeff):
    return expm(-1j*(coeff*s_z+s_x)*dt)

def _diff_matrix_0(coeff):
    return -1j*_matrix_0(coeff)@(s_z*dt)

def _matrix_c_0(coeff):
    return expm(-1j*(coeff*kron(s_z, one) + kron(one, s_z) + kron(s_x, one) + kron(one, s_x) + coeff*kron(s_z-one, s_z-one))*5*dt)

def _diff_matrix_c_0(coeff):
    return -1j*_matrix_c_0(coeff)@((kron(s_z, one) + kron(s_z-one, s_z-one)) * 5*dt)

def _matrix_c_1(coeff):
    return expm(-1j*(kron(s_z, one) + coeff*kron(one, s_z) + kron(s_x, one) + kron(one, s_x) + coeff*kron(s_z-one, s_z-one))*5*dt)

def _diff_matrix_c_1(coeff):
    return -1j*_matrix_c_1(coeff)@((kron(one, s_z) + kron(s_z-one, s_z-one)) *  5*dt)

gate_0 = gene_univ_parameterized_gate('gete_0', _matrix_0, _diff_matrix_0)
gate_c_0 = gene_univ_parameterized_gate('gete_c_0', _matrix_c_0, _diff_matrix_c_0)
gate_c_1 = gene_univ_parameterized_gate('gete_c_1', _matrix_c_1, _diff_matrix_c_1)

circ = Circuit()

circ += gate_0('00').on(0)
circ += gate_0('01').on(0)
circ += gate_0('02').on(0)
circ += gate_0('03').on(0)
circ += gate_0('04').on(0)
circ += gate_0('05').on(0)
circ += gate_0('06').on(0)
circ += gate_0('07').on(0)
circ += gate_0('08').on(0)
circ += gate_0('09').on(0)

circ += gate_0('10').on(1)
circ += gate_0('11').on(1)
circ += gate_0('12').on(1)
circ += gate_0('13').on(1)
circ += gate_0('14').on(1)
circ += gate_0('15').on(1)
circ += gate_0('16').on(1)
circ += gate_0('17').on(1)
circ += gate_0('18').on(1)
circ += gate_0('19').on(1)

circ += gate_c_0('0').on([1,0])
circ += gate_c_0('1').on([1,0])
circ += gate_c_1('2').on([1,0])
circ += gate_c_1('3').on([1,0])

circ += gate_0('010').on(0)
circ += gate_0('011').on(0)
circ += gate_0('012').on(0)
circ += gate_0('013').on(0)
circ += gate_0('014').on(0)
circ += gate_0('015').on(0)
circ += gate_0('016').on(0)
circ += gate_0('017').on(0)
circ += gate_0('018').on(0)
circ += gate_0('019').on(0)

circ += gate_0('110').on(1)
circ += gate_0('111').on(1)
circ += gate_0('112').on(1)
circ += gate_0('113').on(1)
circ += gate_0('114').on(1)
circ += gate_0('115').on(1)
circ += gate_0('116').on(1)
circ += gate_0('117').on(1)
circ += gate_0('118').on(1)
circ += gate_0('119').on(1)

ham = Hamiltonian(QubitOperator('')) 
sim = Simulator('projectq', circ.n_qubits)
sim_left = Simulator('projectq',circ.n_qubits)
grad_ops = sim.get_expectation_with_grad(ham,
                                         circ,
                                         circ_left=Circuit(),
                                         simulator_left=sim_left,
                                         ansatz_params_name=circ.params_name)
lr = 0.02
Quantum_net = MQLayer(grad_ops)
opti = Adam(Quantum_net.trainable_params(), learning_rate=lr)  
net = TrainOneStepCell(Quantum_net, opti)
error_min = 1
error_his = []
for j in range(len(train_x)):
    net(Tensor(train_x[j]), Tensor(train_y[j]))
    params = abs(Quantum_net.weight.asnumpy())
    final_state = []
    for k in range(100): # 100 个测试点
        sim.reset()
        sim.set_qs(eval_x[k])
        sim.apply_circuit(circ, params)
        final_state.append(sim.get_qs())
    error = 1-np.real(np.min([np.abs(np.vdot(bra, ket)) for bra, ket in zip(np.array(final_state), eval_y)]))
    error_his.append(error)
    if error <= error_min:
        error_min = error
        best_params = params
        j_tem = j
    if j % 100 == 0:
        print('error_min:', error_min, 'error_now:', error, 'train_num:', j)
    if error_min < 1e-5:
        break
        
print('\nerror_min:', error_min, 'train_num:', j_tem)
print('\nerror_his:', error_his)
print('best_params:\n', best_params)


error_min: 0.9053189634389555 error_now: 0.9053189634389555 train_num: 0
error_min: 0.23759333043935127 error_now: 0.24801627965004813 train_num: 100
error_min: 0.08266956105074907 error_now: 0.09584347280201366 train_num: 200
error_min: 0.08266956105074907 error_now: 0.14620583396804865 train_num: 300
error_min: 0.06721358103734976 error_now: 0.11502488239076025 train_num: 400
error_min: 0.04832303970476293 error_now: 0.06127615343009429 train_num: 500
error_min: 0.0424617219117186 error_now: 0.052187372640340235 train_num: 600
error_min: 0.0424617219117186 error_now: 0.05538872210227408 train_num: 700
error_min: 0.0424617219117186 error_now: 0.06302231323027763 train_num: 800
error_min: 0.0424617219117186 error_now: 0.08467863010958832 train_num: 900
error_min: 0.0424617219117186 error_now: 0.07154110407069025 train_num: 1000
error_min: 0.0424617219117186 error_now: 0.08528063835450539 train_num: 1100
error_min: 0.0424617219117186 error_now: 0.04741600256952638 train_num: 1200
error_

In [5]:
import numpy as np
import sys
from numpy import kron
from mindquantum import *
from scipy.linalg import expm
import mindspore as ms
from mindspore import ops, Tensor
ms.context.set_context(mode=ms.context.PYNATIVE_MODE, device_target="CPU")
from mindspore.nn import Adam, TrainOneStepCell
from mindspore.common.parameter import Parameter
from mindspore.common.initializer import initializer  
ms.set_seed(1)
np.random.seed(1)

train_x = np.load('./src/cy_train_x.npy', allow_pickle=True)
eval_x = np.load('./src/cy_eval_x.npy', allow_pickle=True)
train_y = np.load('./src/cy_train_y.npy', allow_pickle=True)
eval_y = np.load('./src/cy_eval_y.npy', allow_pickle=True)

s_x = X.matrix()
s_z = Z.matrix()
one = I.matrix()
dt = np.pi/10

def _matrix_0(coeff):
    return expm(-1j*(coeff*s_z+s_x)*dt)

def _diff_matrix_0(coeff):
    return -1j*_matrix_0(coeff)@(s_z*dt)

def _matrix_c_0(coeff):
    return expm(-1j*(coeff*kron(s_z, one) + kron(one, s_z) + kron(s_x, one) + kron(one, s_x) + coeff*kron(s_z-one, s_z-one))*5*dt)

def _diff_matrix_c_0(coeff):
    return -1j*_matrix_c_0(coeff)@((kron(s_z, one) + kron(s_z-one, s_z-one)) * 5*dt)

def _matrix_c_1(coeff):
    return expm(-1j*(kron(s_z, one) + coeff*kron(one, s_z) + kron(s_x, one) + kron(one, s_x) + coeff*kron(s_z-one, s_z-one))*5*dt)

def _diff_matrix_c_1(coeff):
    return -1j*_matrix_c_1(coeff)@((kron(one, s_z) + kron(s_z-one, s_z-one)) *  5*dt)

gate_0 = gene_univ_parameterized_gate('gete_0', _matrix_0, _diff_matrix_0)
gate_c_0 = gene_univ_parameterized_gate('gete_c_0', _matrix_c_0, _diff_matrix_c_0)
gate_c_1 = gene_univ_parameterized_gate('gete_c_1', _matrix_c_1, _diff_matrix_c_1)

circ = Circuit()

circ += gate_0('00').on(0)
circ += gate_0('01').on(0)
circ += gate_0('02').on(0)
circ += gate_0('03').on(0)
circ += gate_0('04').on(0)
circ += gate_0('05').on(0)
circ += gate_0('06').on(0)
circ += gate_0('07').on(0)
circ += gate_0('08').on(0)
circ += gate_0('09').on(0)

circ += gate_0('10').on(1)
circ += gate_0('11').on(1)
circ += gate_0('12').on(1)
circ += gate_0('13').on(1)
circ += gate_0('14').on(1)
circ += gate_0('15').on(1)
circ += gate_0('16').on(1)
circ += gate_0('17').on(1)
circ += gate_0('18').on(1)
circ += gate_0('19').on(1)

circ += gate_c_0('0').on([1,0])
circ += gate_c_0('1').on([1,0])
circ += gate_c_1('2').on([1,0])
circ += gate_c_1('3').on([1,0])

circ += gate_0('010').on(0)
circ += gate_0('011').on(0)
circ += gate_0('012').on(0)
circ += gate_0('013').on(0)
circ += gate_0('014').on(0)
circ += gate_0('015').on(0)
circ += gate_0('016').on(0)
circ += gate_0('017').on(0)
circ += gate_0('018').on(0)
circ += gate_0('019').on(0)

circ += gate_0('110').on(1)
circ += gate_0('111').on(1)
circ += gate_0('112').on(1)
circ += gate_0('113').on(1)
circ += gate_0('114').on(1)
circ += gate_0('115').on(1)
circ += gate_0('116').on(1)
circ += gate_0('117').on(1)
circ += gate_0('118').on(1)
circ += gate_0('119').on(1)

ham = Hamiltonian(QubitOperator('')) 
sim = Simulator('projectq', circ.n_qubits)
sim_left = Simulator('projectq',circ.n_qubits)
grad_ops = sim.get_expectation_with_grad(ham,
                                         circ,
                                         circ_left=Circuit(),
                                         simulator_left=sim_left,
                                         ansatz_params_name=circ.params_name)
lr = 0.05
Quantum_net = MQLayer(grad_ops)
opti = Adam(Quantum_net.trainable_params(), learning_rate=lr)  
net = TrainOneStepCell(Quantum_net, opti)
error_min = 1
error_his = []
for j in range(len(train_x)):
    net(Tensor(train_x[j]), Tensor(train_y[j]))
    params = abs(Quantum_net.weight.asnumpy())
    final_state = []
    for k in range(100): # 100 个测试点
        sim.reset()
        sim.set_qs(eval_x[k])
        sim.apply_circuit(circ, params)
        final_state.append(sim.get_qs())
    error = 1-np.real(np.min([np.abs(np.vdot(bra, ket)) for bra, ket in zip(np.array(final_state), eval_y)]))
    error_his.append(error)
    if error <= error_min:
        error_min = error
        best_params = params
    if j % 100 == 0:
        print('error_min:', error_min, 'error_now:', error, 'train_num:', j)
    if error_min < 1e-6:
        break
        
print('\nerror_min:', error_min, 'train_num:', j_tem)
print('\nerror_his:', error_his)
print('best_params:\n', best_params)


error_min: 0.8295262691050572 error_now: 0.8295262691050572 train_num: 0
error_min: 0.11395823481586054 error_now: 0.14914612277364991 train_num: 100
error_min: 0.05272729766887163 error_now: 0.08646280425881792 train_num: 200
error_min: 0.047177892088439255 error_now: 0.08180193236447453 train_num: 300
error_min: 0.047177892088439255 error_now: 0.12034889004392457 train_num: 400
error_min: 0.047177892088439255 error_now: 0.1796775072852399 train_num: 500
error_min: 0.047177892088439255 error_now: 0.17805890664168522 train_num: 600
error_min: 0.024479784745668698 error_now: 0.053941549917752796 train_num: 700
error_min: 0.024479784745668698 error_now: 0.12065160377277484 train_num: 800
error_min: 0.024479784745668698 error_now: 0.3770434388495816 train_num: 900
error_min: 0.024479784745668698 error_now: 0.27731410231647013 train_num: 1000
error_min: 0.024479784745668698 error_now: 0.18253617038462433 train_num: 1100
error_min: 0.024479784745668698 error_now: 0.14390714189625087 train_n

In [6]:
import numpy as np
import sys
from numpy import kron
from mindquantum import *
from scipy.linalg import expm
import mindspore as ms
from mindspore import ops, Tensor
ms.context.set_context(mode=ms.context.PYNATIVE_MODE, device_target="CPU")
from mindspore.nn import Adam, TrainOneStepCell
from mindspore.common.parameter import Parameter
from mindspore.common.initializer import initializer  
ms.set_seed(1)
np.random.seed(1)

train_x = np.load('./src/cy_train_x.npy', allow_pickle=True)
eval_x = np.load('./src/cy_eval_x.npy', allow_pickle=True)
train_y = np.load('./src/cy_train_y.npy', allow_pickle=True)
eval_y = np.load('./src/cy_eval_y.npy', allow_pickle=True)

s_x = X.matrix()
s_z = Z.matrix()
one = I.matrix()
dt = np.pi/10

def _matrix_0(coeff):
    return expm(-1j*(coeff*s_z+s_x)*dt)

def _diff_matrix_0(coeff):
    return -1j*_matrix_0(coeff)@(s_z*dt)

def _matrix_c_0(coeff):
    return expm(-1j*(coeff*kron(s_z, one) + kron(one, s_z) + kron(s_x, one) + kron(one, s_x) + coeff*kron(s_z-one, s_z-one))*5*dt)

def _diff_matrix_c_0(coeff):
    return -1j*_matrix_c_0(coeff)@((kron(s_z, one) + kron(s_z-one, s_z-one)) * 5*dt)

def _matrix_c_1(coeff):
    return expm(-1j*(kron(s_z, one) + coeff*kron(one, s_z) + kron(s_x, one) + kron(one, s_x) + coeff*kron(s_z-one, s_z-one))*5*dt)

def _diff_matrix_c_1(coeff):
    return -1j*_matrix_c_1(coeff)@((kron(one, s_z) + kron(s_z-one, s_z-one)) *  5*dt)

gate_0 = gene_univ_parameterized_gate('gete_0', _matrix_0, _diff_matrix_0)
gate_c_0 = gene_univ_parameterized_gate('gete_c_0', _matrix_c_0, _diff_matrix_c_0)
gate_c_1 = gene_univ_parameterized_gate('gete_c_1', _matrix_c_1, _diff_matrix_c_1)

circ = Circuit()

circ += gate_0('00').on(0)
circ += gate_0('01').on(0)
circ += gate_0('02').on(0)
circ += gate_0('03').on(0)
circ += gate_0('04').on(0)
circ += gate_0('05').on(0)
circ += gate_0('06').on(0)
circ += gate_0('07').on(0)
circ += gate_0('08').on(0)
circ += gate_0('09').on(0)

circ += gate_0('10').on(1)
circ += gate_0('11').on(1)
circ += gate_0('12').on(1)
circ += gate_0('13').on(1)
circ += gate_0('14').on(1)
circ += gate_0('15').on(1)
circ += gate_0('16').on(1)
circ += gate_0('17').on(1)
circ += gate_0('18').on(1)
circ += gate_0('19').on(1)

circ += gate_c_0('0').on([1,0])
circ += gate_c_0('1').on([1,0])
circ += gate_c_1('2').on([1,0])
circ += gate_c_1('3').on([1,0])

circ += gate_0('010').on(0)
circ += gate_0('011').on(0)
circ += gate_0('012').on(0)
circ += gate_0('013').on(0)
circ += gate_0('014').on(0)
circ += gate_0('015').on(0)
circ += gate_0('016').on(0)
circ += gate_0('017').on(0)
circ += gate_0('018').on(0)
circ += gate_0('019').on(0)

circ += gate_0('110').on(1)
circ += gate_0('111').on(1)
circ += gate_0('112').on(1)
circ += gate_0('113').on(1)
circ += gate_0('114').on(1)
circ += gate_0('115').on(1)
circ += gate_0('116').on(1)
circ += gate_0('117').on(1)
circ += gate_0('118').on(1)
circ += gate_0('119').on(1)

ham = Hamiltonian(QubitOperator('')) 
sim = Simulator('projectq', circ.n_qubits)
sim_left = Simulator('projectq',circ.n_qubits)
grad_ops = sim.get_expectation_with_grad(ham,
                                         circ,
                                         circ_left=Circuit(),
                                         simulator_left=sim_left,
                                         ansatz_params_name=circ.params_name)
lr = 0.1
Quantum_net = MQLayer(grad_ops)
opti = Adam(Quantum_net.trainable_params(), learning_rate=lr)  
net = TrainOneStepCell(Quantum_net, opti)
error_min = 1
error_his = []
for j in range(len(train_x)):
    net(Tensor(train_x[j]), Tensor(train_y[j]))
    params = abs(Quantum_net.weight.asnumpy())
    final_state = []
    for k in range(100): # 100 个测试点
        sim.reset()
        sim.set_qs(eval_x[k])
        sim.apply_circuit(circ, params)
        final_state.append(sim.get_qs())
    error = 1-np.real(np.min([np.abs(np.vdot(bra, ket)) for bra, ket in zip(np.array(final_state), eval_y)]))
    error_his.append(error)
    if error <= error_min:
        error_min = error
        best_params = params
    if j % 100 == 0:
        print('error_min:', error_min, 'error_now:', error, 'train_num:', j)
    if error_min < 1e-6:
        break
        
print('\nerror_min:', error_min, 'train_num:', j_tem)
print('\nerror_his:', error_his)
print('best_params:\n', best_params)


error_min: 0.7986717590827859 error_now: 0.7986717590827859 train_num: 0
error_min: 0.15832549311982513 error_now: 0.37968817825231405 train_num: 100
error_min: 0.1262300347055656 error_now: 0.14490740232562382 train_num: 200
error_min: 0.05548056794431655 error_now: 0.2651106316253432 train_num: 300
error_min: 0.05548056794431655 error_now: 0.2631775940199843 train_num: 400
error_min: 0.05548056794431655 error_now: 0.3094063368976181 train_num: 500
error_min: 0.05548056794431655 error_now: 0.3573245075960275 train_num: 600
error_min: 0.023885727084632435 error_now: 0.0950771324459928 train_num: 700
error_min: 0.023885727084632435 error_now: 0.143350952073519 train_num: 800
error_min: 0.023885727084632435 error_now: 0.22644883404092186 train_num: 900
error_min: 0.023885727084632435 error_now: 0.13216187160624238 train_num: 1000
error_min: 0.023885727084632435 error_now: 0.33042065755261774 train_num: 1100
error_min: 0.023885727084632435 error_now: 0.3105766347967839 train_num: 1200
err

In [7]:
import numpy as np
import sys
from numpy import kron
from mindquantum import *
from scipy.linalg import expm
import mindspore as ms
from mindspore import ops, Tensor
ms.context.set_context(mode=ms.context.PYNATIVE_MODE, device_target="CPU")
from mindspore.nn import Adam, TrainOneStepCell
from mindspore.common.parameter import Parameter
from mindspore.common.initializer import initializer  
ms.set_seed(1)
np.random.seed(1)

train_x = np.load('./src/cy_train_x.npy', allow_pickle=True)
eval_x = np.load('./src/cy_eval_x.npy', allow_pickle=True)
train_y = np.load('./src/cy_train_y.npy', allow_pickle=True)
eval_y = np.load('./src/cy_eval_y.npy', allow_pickle=True)

s_x = X.matrix()
s_z = Z.matrix()
one = I.matrix()
dt = np.pi/10

def _matrix_0(coeff):
    return expm(-1j*(coeff*s_z+s_x)*dt)

def _diff_matrix_0(coeff):
    return -1j*_matrix_0(coeff)@(s_z*dt)

def _matrix_c_0(coeff):
    return expm(-1j*(coeff*kron(s_z, one) + kron(one, s_z) + kron(s_x, one) + kron(one, s_x) + coeff*kron(s_z-one, s_z-one))*5*dt)

def _diff_matrix_c_0(coeff):
    return -1j*_matrix_c_0(coeff)@((kron(s_z, one) + kron(s_z-one, s_z-one)) * 5*dt)

def _matrix_c_1(coeff):
    return expm(-1j*(kron(s_z, one) + coeff*kron(one, s_z) + kron(s_x, one) + kron(one, s_x) + coeff*kron(s_z-one, s_z-one))*5*dt)

def _diff_matrix_c_1(coeff):
    return -1j*_matrix_c_1(coeff)@((kron(one, s_z) + kron(s_z-one, s_z-one)) *  5*dt)

gate_0 = gene_univ_parameterized_gate('gete_0', _matrix_0, _diff_matrix_0)
gate_c_0 = gene_univ_parameterized_gate('gete_c_0', _matrix_c_0, _diff_matrix_c_0)
gate_c_1 = gene_univ_parameterized_gate('gete_c_1', _matrix_c_1, _diff_matrix_c_1)

circ = Circuit()

circ += gate_0('00').on(0)
circ += gate_0('01').on(0)
circ += gate_0('02').on(0)
circ += gate_0('03').on(0)
circ += gate_0('04').on(0)
circ += gate_0('05').on(0)
circ += gate_0('06').on(0)
circ += gate_0('07').on(0)
circ += gate_0('08').on(0)
circ += gate_0('09').on(0)

circ += gate_0('10').on(1)
circ += gate_0('11').on(1)
circ += gate_0('12').on(1)
circ += gate_0('13').on(1)
circ += gate_0('14').on(1)
circ += gate_0('15').on(1)
circ += gate_0('16').on(1)
circ += gate_0('17').on(1)
circ += gate_0('18').on(1)
circ += gate_0('19').on(1)

circ += gate_c_0('0').on([1,0])
circ += gate_c_0('1').on([1,0])
circ += gate_c_1('2').on([1,0])
circ += gate_c_1('3').on([1,0])

circ += gate_0('010').on(0)
circ += gate_0('011').on(0)
circ += gate_0('012').on(0)
circ += gate_0('013').on(0)
circ += gate_0('014').on(0)
circ += gate_0('015').on(0)
circ += gate_0('016').on(0)
circ += gate_0('017').on(0)
circ += gate_0('018').on(0)
circ += gate_0('019').on(0)

circ += gate_0('110').on(1)
circ += gate_0('111').on(1)
circ += gate_0('112').on(1)
circ += gate_0('113').on(1)
circ += gate_0('114').on(1)
circ += gate_0('115').on(1)
circ += gate_0('116').on(1)
circ += gate_0('117').on(1)
circ += gate_0('118').on(1)
circ += gate_0('119').on(1)

ham = Hamiltonian(QubitOperator('')) 
sim = Simulator('projectq', circ.n_qubits)
sim_left = Simulator('projectq',circ.n_qubits)
grad_ops = sim.get_expectation_with_grad(ham,
                                         circ,
                                         circ_left=Circuit(),
                                         simulator_left=sim_left,
                                         ansatz_params_name=circ.params_name)
lr = 0.005
Quantum_net = MQLayer(grad_ops)
opti = Adam(Quantum_net.trainable_params(), learning_rate=lr)  
net = TrainOneStepCell(Quantum_net, opti)
error_min = 1
error_his = []
for j in range(len(train_x)):
    net(Tensor(train_x[j]), Tensor(train_y[j]))
    params = abs(Quantum_net.weight.asnumpy())
    final_state = []
    for k in range(100): # 100 个测试点
        sim.reset()
        sim.set_qs(eval_x[k])
        sim.apply_circuit(circ, params)
        final_state.append(sim.get_qs())
    error = 1-np.real(np.min([np.abs(np.vdot(bra, ket)) for bra, ket in zip(np.array(final_state), eval_y)]))
    error_his.append(error)
    if error <= error_min:
        error_min = error
        best_params = params
    if j % 100 == 0:
        print('error_min:', error_min, 'error_now:', error, 'train_num:', j)
    if error_min < 1e-6:
        break
        
print('\nerror_min:', error_min, 'train_num:', j_tem)
print('\nerror_his:', error_his)
print('best_params:\n', best_params)


error_min: 0.9399304662530873 error_now: 0.9399304662530873 train_num: 0
error_min: 0.5464100094167594 error_now: 0.5572873004504879 train_num: 100
error_min: 0.4214359133663451 error_now: 0.42917085823813483 train_num: 200
error_min: 0.18341599699974764 error_now: 0.18341599699974764 train_num: 300
error_min: 0.1230900386547974 error_now: 0.1285332428860786 train_num: 400
error_min: 0.12083382118575281 error_now: 0.13022215022141292 train_num: 500
error_min: 0.10135720428738637 error_now: 0.10188935039350244 train_num: 600
error_min: 0.09574567871749617 error_now: 0.10169493807835461 train_num: 700
error_min: 0.07186291925248767 error_now: 0.07186291925248767 train_num: 800
error_min: 0.028035592150365884 error_now: 0.028035592150365884 train_num: 900
error_min: 0.02126312579097267 error_now: 0.027380979024830943 train_num: 1000
error_min: 0.02126312579097267 error_now: 0.03397346390470446 train_num: 1100
error_min: 0.02126312579097267 error_now: 0.038168005462750676 train_num: 1200
e

In [8]:
import numpy as np
import sys
from numpy import kron
from mindquantum import *
from scipy.linalg import expm
import mindspore as ms
from mindspore import ops, Tensor
ms.context.set_context(mode=ms.context.PYNATIVE_MODE, device_target="CPU")
from mindspore.nn import Adam, TrainOneStepCell
from mindspore.common.parameter import Parameter
from mindspore.common.initializer import initializer  
ms.set_seed(1)
np.random.seed(1)

train_x = np.load('./src/cy_train_x.npy', allow_pickle=True)
eval_x = np.load('./src/cy_eval_x.npy', allow_pickle=True)
train_y = np.load('./src/cy_train_y.npy', allow_pickle=True)
eval_y = np.load('./src/cy_eval_y.npy', allow_pickle=True)

s_x = X.matrix()
s_z = Z.matrix()
one = I.matrix()
dt = np.pi/10

def _matrix_0(coeff):
    return expm(-1j*(coeff*s_z+s_x)*dt)

def _diff_matrix_0(coeff):
    return -1j*_matrix_0(coeff)@(s_z*dt)

def _matrix_c_0(coeff):
    return expm(-1j*(coeff*kron(s_z, one) + kron(one, s_z) + kron(s_x, one) + kron(one, s_x) + coeff*kron(s_z-one, s_z-one))*5*dt)

def _diff_matrix_c_0(coeff):
    return -1j*_matrix_c_0(coeff)@((kron(s_z, one) + kron(s_z-one, s_z-one)) * 5*dt)

def _matrix_c_1(coeff):
    return expm(-1j*(kron(s_z, one) + coeff*kron(one, s_z) + kron(s_x, one) + kron(one, s_x) + coeff*kron(s_z-one, s_z-one))*5*dt)

def _diff_matrix_c_1(coeff):
    return -1j*_matrix_c_1(coeff)@((kron(one, s_z) + kron(s_z-one, s_z-one)) *  5*dt)

gate_0 = gene_univ_parameterized_gate('gete_0', _matrix_0, _diff_matrix_0)
gate_c_0 = gene_univ_parameterized_gate('gete_c_0', _matrix_c_0, _diff_matrix_c_0)
gate_c_1 = gene_univ_parameterized_gate('gete_c_1', _matrix_c_1, _diff_matrix_c_1)

circ = Circuit()

circ += gate_0('00').on(0)
circ += gate_0('01').on(0)
circ += gate_0('02').on(0)
circ += gate_0('03').on(0)
circ += gate_0('04').on(0)
circ += gate_0('05').on(0)
circ += gate_0('06').on(0)
circ += gate_0('07').on(0)
circ += gate_0('08').on(0)
circ += gate_0('09').on(0)

circ += gate_0('10').on(1)
circ += gate_0('11').on(1)
circ += gate_0('12').on(1)
circ += gate_0('13').on(1)
circ += gate_0('14').on(1)
circ += gate_0('15').on(1)
circ += gate_0('16').on(1)
circ += gate_0('17').on(1)
circ += gate_0('18').on(1)
circ += gate_0('19').on(1)

circ += gate_c_0('0').on([1,0])
circ += gate_c_0('1').on([1,0])
circ += gate_c_1('2').on([1,0])
circ += gate_c_1('3').on([1,0])

circ += gate_0('010').on(0)
circ += gate_0('011').on(0)
circ += gate_0('012').on(0)
circ += gate_0('013').on(0)
circ += gate_0('014').on(0)
circ += gate_0('015').on(0)
circ += gate_0('016').on(0)
circ += gate_0('017').on(0)
circ += gate_0('018').on(0)
circ += gate_0('019').on(0)

circ += gate_0('110').on(1)
circ += gate_0('111').on(1)
circ += gate_0('112').on(1)
circ += gate_0('113').on(1)
circ += gate_0('114').on(1)
circ += gate_0('115').on(1)
circ += gate_0('116').on(1)
circ += gate_0('117').on(1)
circ += gate_0('118').on(1)
circ += gate_0('119').on(1)

ham = Hamiltonian(QubitOperator('')) 
sim = Simulator('projectq', circ.n_qubits)
sim_left = Simulator('projectq',circ.n_qubits)
grad_ops = sim.get_expectation_with_grad(ham,
                                         circ,
                                         circ_left=Circuit(),
                                         simulator_left=sim_left,
                                         ansatz_params_name=circ.params_name)
lr = 0.002
Quantum_net = MQLayer(grad_ops)
opti = Adam(Quantum_net.trainable_params(), learning_rate=lr)  
net = TrainOneStepCell(Quantum_net, opti)
error_min = 1
error_his = []
for j in range(len(train_x)):
    net(Tensor(train_x[j]), Tensor(train_y[j]))
    params = abs(Quantum_net.weight.asnumpy())
    final_state = []
    for k in range(100): # 100 个测试点
        sim.reset()
        sim.set_qs(eval_x[k])
        sim.apply_circuit(circ, params)
        final_state.append(sim.get_qs())
    error = 1-np.real(np.min([np.abs(np.vdot(bra, ket)) for bra, ket in zip(np.array(final_state), eval_y)]))
    error_his.append(error)
    if error <= error_min:
        error_min = error
        best_params = params
    if j % 100 == 0:
        print('error_min:', error_min, 'error_now:', error, 'train_num:', j)
    if error_min < 1e-6:
        break
        
print('\nerror_min:', error_min, 'train_num:', j_tem)
print('\nerror_his:', error_his)
print('best_params:\n', best_params)


error_min: 0.944180376986224 error_now: 0.944180376986224 train_num: 0
error_min: 0.7491241122710053 error_now: 0.7491241122710053 train_num: 100
error_min: 0.5791267836502043 error_now: 0.5791267836502043 train_num: 200
error_min: 0.5601680703577048 error_now: 0.5613390111011416 train_num: 300
error_min: 0.486808634331801 error_now: 0.486808634331801 train_num: 400
error_min: 0.39907716182306296 error_now: 0.39907716182306296 train_num: 500
error_min: 0.32400281454545965 error_now: 0.32400281454545965 train_num: 600
error_min: 0.24081001674598168 error_now: 0.24081001674598168 train_num: 700
error_min: 0.18376364915446008 error_now: 0.18376364915446008 train_num: 800
error_min: 0.1598821086616382 error_now: 0.1598821086616382 train_num: 900
error_min: 0.14423382354683423 error_now: 0.14961144990874076 train_num: 1000
error_min: 0.13899455268351124 error_now: 0.1468637417791011 train_num: 1100
error_min: 0.13899455268351124 error_now: 0.14627932661598253 train_num: 1200
error_min: 0.13

KeyboardInterrupt: 

In [ ]:
import numpy as np
import sys
from numpy import kron
from mindquantum import *
from scipy.linalg import expm
import mindspore as ms
from mindspore import ops, Tensor
ms.context.set_context(mode=ms.context.PYNATIVE_MODE, device_target="CPU")
from mindspore.nn import Adam, TrainOneStepCell
from mindspore.common.parameter import Parameter
from mindspore.common.initializer import initializer  
ms.set_seed(1)
np.random.seed(1)

train_x = np.load('./src/cy_train_x.npy', allow_pickle=True)
eval_x = np.load('./src/cy_eval_x.npy', allow_pickle=True)
train_y = np.load('./src/cy_train_y.npy', allow_pickle=True)
eval_y = np.load('./src/cy_eval_y.npy', allow_pickle=True)

s_x = X.matrix()
s_z = Z.matrix()
one = I.matrix()
dt = np.pi/10

def _matrix_0(coeff):
    return expm(-1j*(coeff*s_z+s_x)*dt)

def _diff_matrix_0(coeff):
    return -1j*_matrix_0(coeff)@(s_z*dt)

def _matrix_c_0(coeff):
    return expm(-1j*(coeff*kron(s_z, one) + kron(one, s_z) + kron(s_x, one) + kron(one, s_x) + coeff*kron(s_z-one, s_z-one))*5*dt)

def _diff_matrix_c_0(coeff):
    return -1j*_matrix_c_0(coeff)@((kron(s_z, one) + kron(s_z-one, s_z-one)) * 5*dt)

def _matrix_c_1(coeff):
    return expm(-1j*(kron(s_z, one) + coeff*kron(one, s_z) + kron(s_x, one) + kron(one, s_x) + coeff*kron(s_z-one, s_z-one))*5*dt)

def _diff_matrix_c_1(coeff):
    return -1j*_matrix_c_1(coeff)@((kron(one, s_z) + kron(s_z-one, s_z-one)) *  5*dt)

gate_0 = gene_univ_parameterized_gate('gete_0', _matrix_0, _diff_matrix_0)
gate_c_0 = gene_univ_parameterized_gate('gete_c_0', _matrix_c_0, _diff_matrix_c_0)
gate_c_1 = gene_univ_parameterized_gate('gete_c_1', _matrix_c_1, _diff_matrix_c_1)

circ = Circuit()

circ += gate_0('00').on(0)
circ += gate_0('01').on(0)
circ += gate_0('02').on(0)
circ += gate_0('03').on(0)
circ += gate_0('04').on(0)
circ += gate_0('05').on(0)
circ += gate_0('06').on(0)
circ += gate_0('07').on(0)
circ += gate_0('08').on(0)
circ += gate_0('09').on(0)

circ += gate_0('10').on(1)
circ += gate_0('11').on(1)
circ += gate_0('12').on(1)
circ += gate_0('13').on(1)
circ += gate_0('14').on(1)
circ += gate_0('15').on(1)
circ += gate_0('16').on(1)
circ += gate_0('17').on(1)
circ += gate_0('18').on(1)
circ += gate_0('19').on(1)

circ += gate_c_0('0').on([1,0])
circ += gate_c_0('1').on([1,0])
circ += gate_c_1('2').on([1,0])
circ += gate_c_1('3').on([1,0])

circ += gate_0('010').on(0)
circ += gate_0('011').on(0)
circ += gate_0('012').on(0)
circ += gate_0('013').on(0)
circ += gate_0('014').on(0)
circ += gate_0('015').on(0)
circ += gate_0('016').on(0)
circ += gate_0('017').on(0)
circ += gate_0('018').on(0)
circ += gate_0('019').on(0)

circ += gate_0('110').on(1)
circ += gate_0('111').on(1)
circ += gate_0('112').on(1)
circ += gate_0('113').on(1)
circ += gate_0('114').on(1)
circ += gate_0('115').on(1)
circ += gate_0('116').on(1)
circ += gate_0('117').on(1)
circ += gate_0('118').on(1)
circ += gate_0('119').on(1)

ham = Hamiltonian(QubitOperator('')) 
sim = Simulator('projectq', circ.n_qubits)
sim_left = Simulator('projectq',circ.n_qubits)
grad_ops = sim.get_expectation_with_grad(ham,
                                         circ,
                                         circ_left=Circuit(),
                                         simulator_left=sim_left,
                                         ansatz_params_name=circ.params_name)
lr = 0.001
Quantum_net = MQLayer(grad_ops)
opti = Adam(Quantum_net.trainable_params(), learning_rate=lr)  
net = TrainOneStepCell(Quantum_net, opti)
error_min = 1
error_his = []
for j in range(len(train_x)):
    net(Tensor(train_x[j]), Tensor(train_y[j]))
    params = abs(Quantum_net.weight.asnumpy())
    final_state = []
    for k in range(100): # 100 个测试点
        sim.reset()
        sim.set_qs(eval_x[k])
        sim.apply_circuit(circ, params)
        final_state.append(sim.get_qs())
    error = 1-np.real(np.min([np.abs(np.vdot(bra, ket)) for bra, ket in zip(np.array(final_state), eval_y)]))
    error_his.append(error)
    if error <= error_min:
        error_min = error
        best_params = params
    if j % 100 == 0:
        print('error_min:', error_min, 'error_now:', error, 'train_num:', j)
    if error_min < 1e-6:
        break
        
print('\nerror_min:', error_min, 'train_num:', j_tem)
print('\nerror_his:', error_his)
print('best_params:\n', best_params)
